# import modules and values

In [1]:
import requests
import zipfile
import io
import os 
import shutil
import pandas as pd
import logging
import configparser
import shutil
import random
import re
import traceback
import yadisk

from pathlib import Path
from datetime import datetime
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine, text, NullPool

import warnings
warnings.filterwarnings("ignore")

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_test']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

# define saving dir

In [4]:
def create_load_save_dir():
    try:
        Path.mkdir(Path.cwd()/'saved_csv')
    except:
        pass
    save_dir = (Path.cwd()/'saved_csv').as_posix()
    return save_dir

In [5]:
local_save_dir = create_load_save_dir()

In [6]:
local_save_dir

'C:/Users/pnknv/Documents/GitHub/yadisk_loader/dev_check_jupyter/saved_csv'

# get list of files already in database table test_avito_parsed

In [7]:
def get_today_date():
    return datetime.strftime(datetime.today(), format = "%d/%m/%Y, %H:%M:%S")

In [8]:
def get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost,
                   localhost_port, database_username, database_password, database_name):
    
    sql_server = SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=(localhost, localhost_port)
    )

    sql_server.start()
    local_port = str(sql_server.local_bind_port)
    sql_engine = create_engine('mariadb+pymysql://{}:{}@{}:{}/{}'.format(database_username, database_password,
                                                                         localhost, local_port, database_name), 
                               poolclass=NullPool)

    return sql_server, sql_engine

In [9]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

# get table of cities

In [10]:
def get_tables_info(engine):
    cities_query='SELECT * FROM city'
    types_query='SELECT * FROM realty_types'
    source_query='SELECT id, name FROM sources'
    district_query = 'SELECT DISTINCT id, city_id, name FROM districts'
    try:
        con_obj=engine.connect()
        city_db = pd.read_sql(text(cities_query), con=con_obj)
        realty_type_db = pd.read_sql(text(types_query), con=con_obj)
        source_db = pd.read_sql(text(source_query), con=con_obj)
        district_db = pd.read_sql(text(district_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        realty_type_db = None
        source_db = None
        district_db = None
        exc_code = exc.code
    return city_db, realty_type_db, source_db, district_db, exc_code

In [11]:
city_df, realty_type_df, source_df, district_df, exc_code = get_tables_info(sql_engine)

In [97]:
exc_code is None

True

In [14]:
city_df.head(2)

,id,name,url_avito,population
0,1,Владивосток,vladivostok,0
1,2,Краснодар,krasnodar,0


In [15]:
realty_type_df

,id,name,rooms
0,1,Комната,1
1,2,Гостинка,1
2,3,1-комнатная,1
3,4,2-комнатная,2
4,5,3-комнатная,3
5,6,4-комнатная,4
6,7,5-комнатная,5
7,8,6 комнат и более,6


In [16]:
source_df

,id,name
0,1,farpost
1,2,cian
2,3,avito


In [17]:
district_df

,id,city_id,name
0,1,1,"64, 71 микрорайоны"
1,2,1,Баляева
2,3,1,БАМ
3,4,1,Борисенко
4,5,1,Весенняя
...,...,...,...
498,499,18,Матвеевка
499,500,18,Южный
500,501,18,Акварельный
501,502,18,Огурцово


# load files

In [18]:
def get_direct_link(sharing_link):
    pk_request = requests.get(ya_api.format(sharing_link))
    # None если не удается преоброазовать ссылку
    return pk_request.json().get('href')

def download_yadisk_files(sharing_link, list_file_exist, save_dir):
    direct_link = get_direct_link(sharing_link)
    if direct_link:
        download = requests.get(direct_link)
        zips = zipfile.ZipFile(io.BytesIO(download.content))
        cnt = 0
        print("files loaded")
        for member in zips.namelist():
            filename = os.path.basename(member)
            if not filename or Path(filename).stem in list_file_exist:
                continue
            src = zips.open(member)
            target = open(os.path.join(save_dir, filename), 'wb')
            with src, target:
                shutil.copyfileobj(src, target)
                cnt += 1
            target.close()
        print('Succesfully downloaded {} files from "{}"'.format(cnt, sharing_link))
        return None
    else:
        print('Failed to download files from "{}"'.format(sharing_link))
        return 'error loading files'

In [19]:
%time
# error_file_loading = download_yadisk_files(ya_link, files_in_db, local_save_dir)

CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
# error_file_loading is None

# upload files to database 

In [21]:
def get_local_files(save_dir):
    p = Path(save_dir).glob('**/*')
    saved_files_list = [x.as_posix() for x in p if x.is_file()]
    return saved_files_list

def create_csv(save_directory):
    saved_files = get_local_files(save_directory)
    cnt_files = 0
    for f_ind in range(len(saved_files)):
        try:
            saved_files_new = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
            saved_files_new['file_name'] = Path(saved_files[f_ind]).stem
            parsed_csv = parsed_csv.append(saved_files_new)
        except:
            parsed_csv = pd.read_csv(saved_files[f_ind], delimiter=';', encoding='utf8')
            parsed_csv['file_name'] = Path(saved_files[f_ind]).stem
        cnt_files += 1
        
    parsed_csv.reset_index(drop=True, inplace=True)
    print('Succesfully opened {} files'.format(cnt_files))

    return parsed_csv

In [22]:
a_full = create_csv(local_save_dir)

Succesfully opened 35 files


In [23]:
a_full

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений,file_name
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 68 м², 16/17 эт.",2-к. квартира,68 м²,16/17 эт.,7999999,117647.0,NaN,р-н Советский,"Сибирская ул., 9А",4 дня назад,Частное лицо,NaN,NaN,NaN,01-03-23 (без дублей)
1,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 25,5 м², 1/4 эт.",2-к. квартира,"25,5 м²",1/4 эт.,1950000,76471.0,NaN,р-н Октябрьский,"Дальне-Ключевская ул., 66",56 минут назад,Агентство,"Агентство недвижимости ""NOVA""",https://www.avito.ru/user/da36df070cc701954bbb...,847,01-03-23 (без дублей)
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 30 м², 1/5 эт.",2-к. квартира,30 м²,1/5 эт.,1870000,62333.0,АЖ GROUP,р-н Октябрьский,"ул. Александра Угрюмова, 1/2",2 часа назад,Агентство,АЖ GROUP,https://www.avito.ru/user/e673859327eeaeee4e57...,NaN,01-03-23 (без дублей)
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 48,8 м², 8/9 эт.",2-к. квартира,"48,8 м²",8/9 эт.,5150000,105533.0,NaN,р-н Ленинский,"Ленинский район, микрорайон Каштак, ул. Ференц...",1 час назад,Частное лицо,NaN,NaN,NaN,01-03-23 (без дублей)
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 40 м², 10/12 эт.",1-к. квартира,40 м²,10/12 эт.,3300000,82500.0,NaN,р-н Октябрьский,"ул. Ивановского, 20",2 часа назад,Агентство,"Агентство недвижимости ""NOVA""",https://www.avito.ru/user/da36df070cc701954bbb...,847,01-03-23 (без дублей)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696381,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 27,7 м², 13/15 эт.",Квартира-студия,"27,7 м²",13/15 эт.,7077350,255500.0,АГАТ-Риэлт,р-н Карасунский,"ул. Уральская, д. 75/6",2 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696382,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 22,1 м², 10/24 эт.",Квартира-студия,"22,1 м²",10/24 эт.,4641000,210000.0,АГАТ-Риэлт,р-н Карасунский,"ул. Новороссийская, лит. 2",1 день назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696383,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 22,5 м², 2/24 эт.",Квартира-студия,"22,5 м²",2/24 эт.,4612500,205000.0,АГАТ-Риэлт,р-н Карасунский,"ул. Новороссийская, лит. 2",3 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)
2696384,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,"2-к. квартира, 45,4 м², 9/9 эт.",2-к. квартира,"45,4 м²",9/9 эт.,5675000,125000.0,АГАТ-Риэлт,р-н Прикубанский,"ул. Западный Обход, д. 42, корп. 2",3 недели назад,Агентство,АГАТ-Риэлт,https://www.avito.ru/user/22ac0a4489a5ad81c070...,NaN,28-02-23 (без дублей)


In [38]:
flist = get_local_files(local_save_dir)

NameError: name 'get_local_files' is not defined

In [25]:
flist

['C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/01-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/02-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/04-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/05-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/06-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-2023.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/07-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/08-02-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/08-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02-23 (без дублей) 2.csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/09-02

# load table avito

In [458]:
a = pd.read_csv('23-03-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [459]:
a

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 36,6 м², 7/9 эт.",2-к. квартира,"36,6 м²",7/9 эт.,2600000,71038.0,NaN,р-н Кировский,"Учебная ул., 15",43 минуты назад,Частное лицо,NaN,NaN,NaN
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 18 м², 3/4 эт.",1-к. квартира,18 м²,3/4 эт.,1900000,105556.0,NaN,р-н Кировский,"ул. Усова, 66",2 часа назад,Агентство,Перспектива24-Томск,https://www.avito.ru/user/b1f3c879b3fd07d3d782...,NaN
2,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 36,6 м², 10/10 эт.",1-к. квартира,"36,6 м²",10/10 эт.,3940000,107650.0,NaN,р-н Советский,"ул. Елизаровых, 76",2 часа назад,Частное лицо,NaN,NaN,NaN
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 52 м², 2/5 эт.",2-к. квартира,52 м²,2/5 эт.,4500000,86538.0,NaN,р-н Ленинский,ул. Интернационалистов,1 час назад,Частное лицо,NaN,NaN,NaN
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 29,7 м², 3/5 эт.",1-к. квартира,"29,7 м²",3/5 эт.,3600000,121212.0,NaN,р-н Кировский,"ул. Косарева, 25",2 часа назад,Агентство,Этажи Томск,https://www.avito.ru/user/2abecfd498ab128191d8...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129602,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,1 м², 5/16 эт.",Квартира-студия,"24,1 м²",5/16 эт.,4097000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",6 дней назад,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN
129603,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,1 м², 4/16 эт.",Квартира-студия,"24,1 м²",4/16 эт.,4097000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",3 недели назад,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN
129604,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,4 м², 2/16 эт.",Квартира-студия,"24,4 м²",2/16 эт.,4148000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",2 дня назад,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN
129605,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,4 м², 2/16 эт.",Квартира-студия,"24,4 м²",2/16 эт.,4148000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",2 дня назад,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN


In [460]:
# a = pd.read_csv(flist[4], delimiter=';', encoding='utf8')

In [461]:
# a = pd.read_csv('saved_avito_csv/20-03-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [462]:
index_to_drop = a[(a['Площадь'].isna()) | (a['Заголовок'] == a['Комнат']) | (a['Цена'].isna())].index.tolist()
if len(index_to_drop) != 0:
    a.drop(index_to_drop, inplace=True)
a.drop_duplicates(subset=['Ссылка'], keep='last', inplace=True)
a.reset_index(drop=True, inplace=True)

# dev realty avito

In [463]:
# add city

In [464]:
a['city'] = a['Ссылка'].apply(lambda x: x.split('/')[3])

In [465]:
# source_id

In [466]:
source_df_dict = source_df.set_index('id').to_dict()['name']

In [467]:
def source_id_from_link(link_value, source_dict):
    try:
        return list(source_dict.keys())[list(source_dict.values()).index(link_value.split('.')[1])]
    except:
        return None

In [468]:
a['source_id'] = a['Ссылка'].apply(lambda x: source_id_from_link(x, source_df_dict))

In [469]:
# ad_id

In [470]:
a['Ссылка'].apply(lambda x: x.split('_')[-1])

0         2941699896
1         2768075746
2         2855870978
3         2918490165
4         2830396463
             ...    
123836    2456436885
123837    2455914517
123838    2455996553
123839    2456764040
123840    2455906412
Name: Ссылка, Length: 123841, dtype: object

In [471]:
a['ad_id'] = a['Ссылка'].apply(lambda x: x.split('_')[-1])

In [472]:
a[a.ad_id.apply(lambda x: len(x) == 9)].ad_id.astype(int)

144      862868293
295      696712852
361      886798235
1007     591471963
1069     964496447
           ...    
41025    982283456
41595    989205424
42571    992580088
43267    986559150
51071    733105956
Name: ad_id, Length: 143, dtype: int32

In [473]:
# city_id

In [474]:
city_df_dict = city_df.set_index('id').to_dict()['url_avito']

In [475]:
def city_id_from_link(city_value, city_dict):
    try:
        return int(list(city_dict.keys())[list(city_dict.values()).index(city_value)])
    except Exception as e:
        return None

In [476]:
a['city_id'] = a['city'].apply(lambda x: city_id_from_link(x, city_df_dict))
# удалить записи с лишними городами и преобразовать в int
# a.dropna(subset=['city_id'], inplace=True)
# a.reset_index(drop=True, inplace=True)
# a.city_id = a.city_id.astype(int)

In [477]:
for i, val in a.iterrows():
    if val.city_id == None:
        print(i)

In [107]:
# district_id будет удален из таблицы

In [108]:
a['district_id'] = None

In [109]:
# type_id

In [110]:
dict_realty_type = {
    '1': ['Комната'],
    '2': ['Квартира-студия', 'Апартаменты-студия', 'Гостинка', 'Студия'],
    '3': ['1-к. квартира', '1-к. апартаменты', 'Своб. планировка', '1-комнатная', '1 комн', 'Свободная планировка'],
    '4': ['2-к. квартира', '2-к. апартаменты', '2-комнатная', '2 комн'],
    '5': ['3-к. квартира', '3-к. апартаменты', '3-комнатная', '3 комн'],
    '6': ['4-к. квартира', '4-к. апартаменты', '4-комнатная', '4 комн'],
    '7': ['5-к. квартира', '5-к. апартаменты', '5-комнатная', '5 комн'],
    '8': ["6 комнат и более", '6 комн'],
}
# 9 аукцион, доля

In [111]:
def type_id_from_room(room_value, type_dict):
    tid = None
    try:
        for ind, values in type_dict.items():
            if room_value in values:
                tid = ind
                return tid
        if any(elem in room_value for elem in ['Доля', 'Аукцион']):
            # tid = 9 # аукцион
            return None
        elif all(elem in room_value for elem in ['Комната ', ' в ', '-к']):
            tid = 1 # комната в квартире
            return tid
        elif any(elem in room_value for elem in ['6-к.', '7-к.', '8-к.', '9-к.', '10 и более-к.', '10 комнат и больше']):
            tid = 8 # 6 комнат и более
            return tid
        else:
            return None # none
    except:
        return None

In [112]:
a['type_id'] = a['Комнат'].apply(lambda x: type_id_from_room(x, dict_realty_type))
# удалить записи аукционов и других объяв где не опредлен type_id, преобразовать цены в int
a.dropna(subset=['type_id'], inplace=True)
a.reset_index(drop=True, inplace=True)
a['Цена'] = a['Цена'].astype(float)

In [113]:
a

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,...,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений,city,source_id,ad_id,city_id,district_id,type_id
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 36,6 м², 7/9 эт.",2-к. квартира,"36,6 м²",7/9 эт.,2600000.0,71038.0,NaN,р-н Кировский,"Учебная ул., 15",...,Частное лицо,NaN,NaN,NaN,tomsk,3,2941699896,17,None,4
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 18 м², 3/4 эт.",1-к. квартира,18 м²,3/4 эт.,1900000.0,105556.0,NaN,р-н Кировский,"ул. Усова, 66",...,Агентство,Перспектива24-Томск,https://www.avito.ru/user/b1f3c879b3fd07d3d782...,NaN,tomsk,3,2768075746,17,None,3
2,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 36,6 м², 10/10 эт.",1-к. квартира,"36,6 м²",10/10 эт.,3940000.0,107650.0,NaN,р-н Советский,"ул. Елизаровых, 76",...,Частное лицо,NaN,NaN,NaN,tomsk,3,2855870978,17,None,3
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 52 м², 2/5 эт.",2-к. квартира,52 м²,2/5 эт.,4500000.0,86538.0,NaN,р-н Ленинский,ул. Интернационалистов,...,Частное лицо,NaN,NaN,NaN,tomsk,3,2918490165,17,None,4
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 29,7 м², 3/5 эт.",1-к. квартира,"29,7 м²",3/5 эт.,3600000.0,121212.0,NaN,р-н Кировский,"ул. Косарева, 25",...,Агентство,Этажи Томск,https://www.avito.ru/user/2abecfd498ab128191d8...,NaN,tomsk,3,2830396463,17,None,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123704,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,1 м², 5/16 эт.",Квартира-студия,"24,1 м²",5/16 эт.,4097000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",...,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN,krasnodar,3,2456436885,2,None,2
123705,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,1 м², 4/16 эт.",Квартира-студия,"24,1 м²",4/16 эт.,4097000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",...,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN,krasnodar,3,2455914517,2,None,2
123706,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,4 м², 2/16 эт.",Квартира-студия,"24,4 м²",2/16 эт.,4148000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",...,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN,krasnodar,3,2455996553,2,None,2
123707,https://www.avito.ru/krasnodar/kvartiry/kvarti...,"Квартира-студия, 24,4 м², 2/16 эт.",Квартира-студия,"24,4 м²",2/16 эт.,4148000.0,170000.0,DOGMA,р-н Прикубанский,"ул. Западный Обход, стр. 27",...,Застройщик,DOGMA,https://www.avito.ru/user/5e54ceaee17b94b9e75f...,NaN,krasnodar,3,2456764040,2,None,2


In [114]:
# addr

In [115]:
# создание поля addr из данных avito
# поле должно совпасть с форматом cian
# формат: Томская область; Томск; р-н Кировский; мкр. Преображенский; улица Дзержинского; 36
def avito_addr_from_row(row, city_df):
    # добавление города для совпадения форматов
    try:
        addr = city_df[city_df.url_avito == row['city']]['name'].values[0]
        addr += '; '
    except:
        addr = ''
    # добавляем район, в авито он может быть None
    try:
        if len(row['Район']) == 0:
            pass
        else:
            addr += row['Район']
            addr += '; '
    except:
        pass
    # добавляем улицу, дом, пр. в авито он может быть None
    try:
        for value in row['Улица'].split(', '):
            addr += value
            addr += '; '
    except:
        pass
    # drop duplicates
    addr = addr[:-2]
    if len(addr.split('; ')) != len(set(addr.split('; '))):
        addr = '; '.join(list(dict.fromkeys(addr.split('; '))))
    return addr

In [116]:
a['addr'] = a.apply(lambda row: avito_addr_from_row(row, city_df), axis=1)

In [117]:
# square

In [118]:
def square_from_ploshad(square_value):
    try:
        return float(square_value[:-3].replace(',', '.'))
    except:
        return None

In [119]:
a['square'] = a['Площадь'].apply(lambda x: square_from_ploshad(x))

In [120]:
# floor and house_floors

In [121]:
def floor_floors_from_etazh(etazh_value):
    try:
        return [etazh_value.split('/')[0], etazh_value.split('/')[1].replace(' эт.', '')]
    except:
        return [None, None]

In [122]:
a['floor'],  a['house_floors'] = zip(*a['Этаж'].apply(floor_floors_from_etazh))

In [123]:
# link

In [124]:
a['link'] = a['Ссылка']

In [125]:
# date

In [126]:
files_to_process_avito = ['23-03-23 (без дублей).csv']

In [127]:
def get_date_from_name(fname):
    try:
        return datetime.strptime(fname.replace("(без дублей)", "").replace("циан", "").replace(" ", ""), "%d-%m-%y")
    except:
        return datetime.strptime(Path(fname).stem.replace("(без дублей)", "").replace("циан", "").replace(" ", ""),
                                 "%d-%m-%y")

In [128]:
a['date'] = get_date_from_name(Path(files_to_process_avito[0]).stem)

In [129]:
# status

In [130]:
a['status'] = None

In [131]:
# version

In [132]:
def get_version(engine, today_date, source_id):
    last_version = f"SELECT Max(version) FROM realty WHERE date < '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [133]:
current_source = a.source_id.unique()[0]
current_date = datetime.now().date()
last_version = get_version(sql_engine, current_date, current_source)[0].iloc[0,0]
if last_version == None:
    current_version = 1
else:
    current_version = last_version + 1

In [134]:
a['version'] = current_version

In [135]:
# offer_from
# ВОПРОС - надо ли преобразовать к одному виду с cian

In [136]:
a['offer_from'] = a['Тип продавца']

In [137]:
# status_new

In [138]:
a['status_new'] = 1

In [139]:
# house_id
# нет в текущем алго, оставляю пустым

In [194]:
realty_table = a[['source_id', 'ad_id', 'city_id', 'district_id', 'type_id', 'addr', 'square', 'floor', 'house_floors', 'link', 'date', 'status', 'version', 'offer_from', 'status_new']]

# check and load dev realty

In [195]:
realty_table_check = realty_table[['source_id', 'ad_id', 'city_id', 'type_id', 'addr', 'square', 'floor', 'house_floors', 'link', 'date', 'version', 'offer_from', 'status_new']]

In [196]:
# удаление строк где одно из важных значений пустое
na_ind = realty_table_check[realty_table_check.isna().any(axis=1)].index
if len(na_ind) > 0:
    realty_table = realty_table.drop(na_ind)

удалено 1 строк


In [206]:
len(realty_table)

123708

In [347]:
# st_time = datetime.now()
# realty_table.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
# print(len(realty_table), datetime.now() - st_time)

119493 0:01:45.546200


# dev prices

In [249]:
# realty_id

In [250]:
def get_realty_ids(engine, today_date, source_id):
    id_query = f"SELECT link, id AS 'realty_id' FROM realty WHERE date = '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        id_db = pd.read_sql(text(id_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        id_db = None
        exc_code = exc.code
    return id_db, exc_code

In [251]:
files_to_process_avito = ['23-03-23 (без дублей).csv']

In [252]:
current_source = a.source_id.unique()[0]
current_date = datetime.now().date()
avito_prices = get_realty_ids(sql_engine, get_date_from_name(files_to_process_avito[0]), current_source)[0]

In [253]:
avito_prices

,link,realty_id
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,819898
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819899
2,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819900
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,819901
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819902
...,...,...
123704,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943602
123705,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943603
123706,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943604
123707,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943605


In [254]:
# cian_prices['realty_id'] = cian_realty['link'].map(id_df.set_index('link')['id'])

In [255]:
# price

In [256]:
avito_prices=pd.merge(avito_prices,a[['link', 'Цена']], on='link', how='left').rename(columns={'Цена': 'price'})

In [106]:
# price_sqrm

In [258]:
avito_prices=pd.merge(avito_prices,a[['link', 'Цена за м2']], on='link', how='left').rename(columns={'Цена за м2': 'price_sqrm'})

In [111]:
# date

In [260]:
avito_prices['date'] = get_date_from_name(files_to_process_avito[0])

In [261]:
avito_prices

,link,realty_id,price,price_sqrm,date
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,819898,2600000.0,71038.0,2023-03-23
1,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819899,1900000.0,105556.0,2023-03-23
2,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819900,3940000.0,107650.0,2023-03-23
3,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,819901,4500000.0,86538.0,2023-03-23
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,819902,3600000.0,121212.0,2023-03-23
...,...,...,...,...,...
123704,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943602,4097000.0,170000.0,2023-03-23
123705,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943603,4097000.0,170000.0,2023-03-23
123706,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943604,4148000.0,170000.0,2023-03-23
123707,https://www.avito.ru/krasnodar/kvartiry/kvarti...,943605,4148000.0,170000.0,2023-03-23


In [87]:
# estimate_min estimate_max estimate_med estimate_sqrm diviation skip fill in default values of sql table

In [110]:
# avito_prices[['estimate_min', 'estimate_max', 'estimate_med', 'estimate_sqrm', 'diviation']] = None

In [113]:
# status skip fill in default values of sql table

In [92]:
# avito_prices['status'] = None

In [108]:
# same_cnt same_urls is_actual leftt empty to fill in default values

In [114]:
# avito_prices[['same_cnt', 'same_urls', 'is_actual']] = None

# check and load dev prices

In [115]:
avito_prices

,realty_id,price,price_sqrm,date
0,NaN,4800000,81633.0,2023-03-21
1,NaN,4650000,106651.0,2023-03-21
2,NaN,7700000,85842.0,2023-03-21
3,NaN,4600000,121053.0,2023-03-21
4,NaN,3690000,95596.0,2023-03-21
...,...,...,...,...
119488,NaN,7200000,101408.0,2023-03-21
119489,NaN,6400000,101587.0,2023-03-21
119490,NaN,19350500,236270.0,2023-03-21
119491,NaN,6097000,130000.0,2023-03-21


In [508]:
st_time = datetime.now()
avito_prices.to_sql(name='prices', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
print(len(avito_prices), datetime.now() - st_time)

119493 0:00:24.363198


# cian load

In [515]:
cian_all = pd.read_csv('24-03-23.csv', delimiter=',', encoding='cp1251')

In [516]:
cian_all.drop_duplicates(subset=['link'], keep='last', inplace=True)

# dev realty cian

In [517]:
cian_realty = pd.DataFrame()

In [518]:
# source_id

In [519]:
source_df_dict = source_df.set_index('id').to_dict()['name']

In [520]:
def source_id_from_link_cian(link_value, source_dict):
    try:
        return list(source_dict.keys())[list(source_dict.values()).index(link_value)]
    except:
        return None

In [521]:
cian_realty['source_id'] = cian_all['source'].apply(lambda x: source_id_from_link_cian(x, source_df_dict))

In [522]:
# ad_id

In [523]:
cian_realty['ad_id'] = cian_all['ad_id']

In [524]:
# city_id

In [525]:
city_df_dict = city_df.set_index('id').to_dict()['name']

In [526]:
def city_id_from_link(city_value, city_dict):
    try:
        return int(list(city_dict.keys())[list(city_dict.values()).index(city_value)])
    except Exception as e:
        return None

In [527]:
cian_realty['city_id'] = cian_all['city'].apply(lambda x: city_id_from_link(x, city_df_dict))
# удалить записи где city_id пустой (записи городов, которые не присутствуют в списке городов для парсинга)
# ind_to_drop = cian_realty[(cian_realty['city_id'].isna())].index.tolist()
# cian_realty.drop(ind_to_drop, inplace=True)
# cian_all.drop(ind_to_drop, inplace=True)
# cian_realty.city_id = cian_realty.city_id.astype(int)

In [528]:
# district_id пока None

In [529]:
cian_realty['district_id'] = None

In [530]:
# type_id

In [531]:
dict_realty_type = {
    '1': ['Комната'],
    '2': ['Квартира-студия', 'Апартаменты-студия', 'Гостинка', 'Студия'],
    '3': ['1-к. квартира', '1-к. апартаменты', 'Своб. планировка', '1-комнатная', '1 комн', 'Свободная планировка'],
    '4': ['2-к. квартира', '2-к. апартаменты', '2-комнатная', '2 комн'],
    '5': ['3-к. квартира', '3-к. апартаменты', '3-комнатная', '3 комн'],
    '6': ['4-к. квартира', '4-к. апартаменты', '4-комнатная', '4 комн'],
    '7': ['5-к. квартира', '5-к. апартаменты', '5-комнатная', '5 комн'],
    '8': ["6 комнат и более", '6 комн'],
}
# 9 аукцион, доля

In [532]:
def type_id_from_room(room_value, type_dict):
    tid = None
    try:
        for ind, values in type_dict.items():
            if room_value in values:
                tid = ind
                return tid
        if any(elem in room_value for elem in ['Доля', 'Аукцион']):
            # tid = 9 # аукцион
            return None
        elif all(elem in room_value for elem in ['Комната ', ' в ', '-к']):
            tid = 1 # комната в квартире
            return tid
        elif any(elem in room_value for elem in ['6-к.', '7-к.', '8-к.', '9-к.', '10 и более-к.', '10 комнат и больше']):
            tid = 8 # 6 комнат и более
            return tid
        else:
            return None # none
    except:
        return None

In [534]:
cian_realty['type_id'] = cian_all['type'].apply(lambda x: type_id_from_room(x, dict_realty_type))
# удалить записи аукционов и других объяв где не опредлен type_id
ind_to_drop = cian_realty[(cian_realty['type_id'].isna())].index.tolist()
cian_realty.drop(ind_to_drop, inplace=True)
cian_all.drop(ind_to_drop, inplace=True)
cian_realty.type_id = cian_realty.type_id.astype(int)

In [535]:
# addr

In [536]:
cian_realty['addr'] = cian_all['addr']

In [537]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr
0,2,210197059,5.0,None,5,Краснодарский край; Анапа; улица Горького; 2А
1,2,213575526,5.0,None,5,Краснодарский край; Анапа; улица Краснозеленых...
2,2,232012292,5.0,None,2,Краснодарский край; Анапа; улица Ленина; 194
3,2,233965925,5.0,None,3,Краснодарский край; Анапа; улица Толстого; 81
4,2,239179225,5.0,None,7,Краснодарский край; Анапа; Владимирская улица;...
...,...,...,...,...,...,...
103641,2,285278480,17.0,None,4,Томская область; Томск; р-н Октябрьский; мкр. ...
103642,2,285278483,17.0,None,4,Томская область; Томск; р-н Октябрьский; мкр. ...
103643,2,285278702,17.0,None,5,Томская область; Томск; р-н Октябрьский; улица...
103644,2,285280917,17.0,None,2,Томская область; Томск; р-н Советский; Заливна...


In [538]:
# square

In [539]:
cian_realty['square'] = cian_all['square']

In [540]:
# floor and house_floors

In [541]:
cian_realty['floor'],  cian_realty['house_floors'] = cian_all['floor'], cian_all['house_floors']

In [542]:
# link

In [543]:
cian_realty['link'] = cian_all['link']

In [544]:
# date

In [545]:
cian_realty['date'] = datetime.now().date() # make date from filename format 21-03-2023

In [546]:
# status

In [547]:
cian_realty['status'] = None

In [548]:
# version

In [551]:
def get_version(engine, today_date, source_id):
    last_version = f"SELECT Max(version) FROM realty WHERE date < '{today_date}' AND source_id = {source_id}"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        last_version_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [552]:
current_source = cian_realty.source_id.unique()[0]
current_date = datetime.now().date()
last_version = get_version(sql_engine, current_date, current_source)[0].iloc[0,0]
if last_version == None:
    current_version = 1
else:
    current_version = last_version + 1

(pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


AttributeError: 'NoneType' object has no attribute 'iloc'

In [554]:
current_version_ = 272

In [555]:
cian_realty['version'] = current_version_

In [556]:
# offer_from

In [557]:
cian_realty['offer_from'] = cian_all['offer_from']

In [558]:
# status_new

In [559]:
cian_realty['status_new'] = 1

In [560]:
# house_id
# нет в текущем алго, оставляю пустым

In [561]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5.0,None,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-26,None,272,Собственник,1
1,2,213575526,5.0,None,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-26,None,272,Риелтор,1
2,2,232012292,5.0,None,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-26,None,272,Риелтор,1
3,2,233965925,5.0,None,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-26,None,272,Агентство недвижимости,1
4,2,239179225,5.0,None,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-26,None,272,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103641,2,285278480,17.0,None,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,5,5,https://tomsk.cian.ru/sale/flat/285278480/,2023-03-26,None,272,Агентство недвижимости,1
103642,2,285278483,17.0,None,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,2,5,https://tomsk.cian.ru/sale/flat/285278483/,2023-03-26,None,272,Агентство недвижимости,1
103643,2,285278702,17.0,None,5,Томская область; Томск; р-н Октябрьский; улица...,51.0,2,5,https://tomsk.cian.ru/sale/flat/285278702/,2023-03-26,None,272,Агентство недвижимости,1
103644,2,285280917,17.0,None,2,Томская область; Томск; р-н Советский; Заливна...,19.6,1,4,https://tomsk.cian.ru/sale/flat/285280917/,2023-03-26,None,272,Агентство недвижимости,1


# districts

In [562]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_test']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

In [563]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [565]:
def get_districts(engine):
    districts_query = f"SELECT DISTINCT id, city_id, name FROM districts"
    try:
        con_obj=engine.connect()
        districts_query_db = pd.read_sql(text(districts_query), con=con_obj)
        con_obj.close()
    except Exception as exc:
        print(exc)
        districts_query_db = None
    return districts_query_db

In [580]:
pd.is

NaT

In [593]:
def district_from_rn_mkrn(realty_row, all_districts, sql_engine):
    # для записи новых районов создадим датафрейм и получим последний id района
    add_districts_df = pd.DataFrame(columns=['city_id', 'name'])
    current_max_id = all_districts.id.max()
    # для работы функции создадим объекты текущего р-н и мкр
    current_rn = None
    current_mkrn = None
    if not np.isnan(realty_row['city_id']):
        for addr_part in realty_row.addr.split(';')[:4]:
            if 'мкр.' in addr_part:
                current_mkrn = addr_part.replace(' мкр. ', '')
                continue
            elif 'р-н' in addr_part:
                current_rn = addr_part.replace(' р-н ', '')
                continue
            else:
                continue
        if current_rn == None and current_mkrn == None:
            return None
        else:
            district_name = current_mkrn if current_mkrn != None else current_rn
            district_intersection = all_districts[
                (all_districts.city_id == realty_row.city_id) & (all_districts.name == district_name)]
            if len(district_intersection) == 0:
                # обновление данных из districts на случай если дистрикт добавлялся во время исполнения скрипта
                all_districts_upd = get_districts(sql_engine)
                district_intersection_upd = all_districts_upd[
                    (all_districts_upd.city_id == realty_row.city_id) & (all_districts_upd.name == district_name)]
                current_max_id = int(all_districts_upd.id.max())
                # если в обновленном districts нет данных о районе - добавляем
                if len(district_intersection_upd) == 0:
                    print(district_intersection_upd)
                    print(realty_row['city_id'].isna())
                    add_districts_df.loc[len(add_districts_df)] = [realty_row.city_id, district_name]
                    add_districts_df.to_sql(name='districts', con=sql_engine, if_exists='append', chunksize=7000,
                                            method='multi', index=False)
                    print('добавлен новый район "{}" в districts функцией district_from_rn_mkrn'.format(district_name))
                    return current_max_id + 1
                # если данные о районе есть, возвращаем id
                else:
                    return district_intersection_upd.iloc[0, 0]
            else:
                return district_intersection.iloc[0, 0]
    else:
        return None

In [594]:
all_districts = get_districts(sql_engine)

In [595]:
cian_realty['district_id'] = cian_realty.apply(lambda row: district_from_rn_mkrn(row, all_districts, sql_engine), axis=1)

In [598]:
cian_realty.iloc[28619:28620,:]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
28619,2,270222384,NaN,NaN,6,Краснодарский край; Сириус пгт; р-н Адлерский;...,127.0,2,3,https://krasnodar.cian.ru/sale/flat/270222384/,2023-03-26,None,272,Агентство недвижимости,1


In [ ]:
cian_realty.iloc[]

In [596]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-26,None,272,Собственник,1
1,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-26,None,272,Риелтор,1
2,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-26,None,272,Риелтор,1
3,2,233965925,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-26,None,272,Агентство недвижимости,1
4,2,239179225,5.0,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-26,None,272,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103641,2,285278480,17.0,421.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,5,5,https://tomsk.cian.ru/sale/flat/285278480/,2023-03-26,None,272,Агентство недвижимости,1
103642,2,285278483,17.0,421.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,2,5,https://tomsk.cian.ru/sale/flat/285278483/,2023-03-26,None,272,Агентство недвижимости,1
103643,2,285278702,17.0,391.0,5,Томская область; Томск; р-н Октябрьский; улица...,51.0,2,5,https://tomsk.cian.ru/sale/flat/285278702/,2023-03-26,None,272,Агентство недвижимости,1
103644,2,285280917,17.0,384.0,2,Томская область; Томск; р-н Советский; Заливна...,19.6,1,4,https://tomsk.cian.ru/sale/flat/285280917/,2023-03-26,None,272,Агентство недвижимости,1


In [79]:
all_districts = get_districts(sql_engine)

In [80]:
cian_realty.district_id = cian_realty.apply(lambda row: district_from_rn_mkrn(row, all_districts, sql_engine), axis=1)

In [82]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,2,285075402,17,384.0,3,Томская область; Томск; р-н Советский; проспек...,18.0,5,5,https://tomsk.cian.ru/sale/flat/285075402/,2023-03-24,None,270,Агентство недвижимости,1
101776,2,285075572,17,384.0,3,Томская область; Томск; р-н Советский; улица Ш...,16.5,2,5,https://tomsk.cian.ru/sale/flat/285075572/,2023-03-24,None,270,Агентство недвижимости,1
101777,2,285076163,17,394.0,4,Томская область; Томск; р-н Ленинский; улица Р...,48.0,5,6,https://tomsk.cian.ru/sale/flat/285076163/,2023-03-24,None,270,Агентство недвижимости,1
101778,2,285076291,17,506.0,3,Томская область; Томск; р-н Уебский лол; улица...,39.0,4,10,https://tomsk.cian.ru/sale/flat/285076291/,2023-03-24,None,270,Агентство недвижимости,1


In [ ]:
# проверка если есть строковые значения в 

In [252]:
def find_str(x):
    try:
        _ = int(x)
        return False
    except:
        return True

In [253]:
cian_realty[(~cian_realty.district_id.isna()) & (cian_realty.district_id.apply(lambda x: tryx(x)))]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new


In [220]:
cian_realty

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,2,285075402,17,384.0,3,Томская область; Томск; р-н Советский; проспек...,18.0,5,5,https://tomsk.cian.ru/sale/flat/285075402/,2023-03-24,None,270,Агентство недвижимости,1
101776,2,285075572,17,384.0,3,Томская область; Томск; р-н Советский; улица Ш...,16.5,2,5,https://tomsk.cian.ru/sale/flat/285075572/,2023-03-24,None,270,Агентство недвижимости,1
101777,2,285076163,17,394.0,4,Томская область; Томск; р-н Ленинский; улица Р...,48.0,5,6,https://tomsk.cian.ru/sale/flat/285076163/,2023-03-24,None,270,Агентство недвижимости,1
101778,2,285076291,17,394.0,3,Томская область; Томск; р-н Ленинский; Черемош...,39.0,4,10,https://tomsk.cian.ru/sale/flat/285076291/,2023-03-24,None,270,Агентство недвижимости,1


2023-03-24 16:47:03,158| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [203]:
def test_rn_mkrn(realty_row, all_districts):
    current_rn = None
    current_mkrn = None
    for addr_part in realty_row.addr.split(';')[:3]:
        if 'мкр.' in addr_part:
            current_mkrn = addr_part.replace(' мкр. ', '')
        elif 'р-н' in addr_part:
            current_rn = addr_part.replace(' р-н ', '')
        else:
            pass
    if current_rn == None and current_mkrn == None:
        return None
    else:
        district_name = current_mkrn if current_mkrn != None else current_rn
        print(current_rn)
        print(current_mkrn)
        print(district_name)
        district_intersection = all_districts[(all_districts.city_id == realty_row.city_id) & (all_districts.name == district_name)]
        if len(district_intersection) == 0:
            return district_name
            # add row to new_district_df
        else:
            print('asd')
            return district_intersection.iloc[0,0]

In [206]:
cian_realty.iloc[-1,:]

source_id                                                       2
ad_id                                                   285076712
city_id                                                        17
district_id                                             Советский
type_id                                                         4
addr            Томская область; Томск; р-н Советский; улица К...
square                                                       46.0
floor                                                           4
house_floors                                                    5
link                   https://tomsk.cian.ru/sale/flat/285076712/
date                                                   2023-03-24
status                                                       None
version                                                       270
offer_from                                                Риелтор
status_new                                                      1
Name: 1017

In [204]:
print(test_rn_mkrn(cian_realty.iloc[-1,:], all_districts))

 Советский
None
 Советский
 Советский


In [208]:
all_districts[all_districts.city_id == 17].iloc[2,2]

'Советский'

In [190]:
def tryx(x):
    try:
        _ = int(x)
        return True
    except:
        return False

In [191]:
cian_realty[cian_realty.district_id.apply(lambda x: tryx(x))]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new


In [ ]:
a['floor'],  a['house_floors'] = zip(*a['Этаж'].apply(floor_floors_from_etazh))

In [89]:
cian_rm.addr.apply(lambda x: 'р-н' in x.split(';')[2])

8576      True
8577      True
8578      True
8579      True
8580      True
          ... 
101775    True
101776    True
101777    True
101778    True
101779    True
Name: addr, Length: 66956, dtype: bool

In [90]:
~cian_rm.raion.isna()

8576      True
8577      True
8578      True
8579      True
8580      True
          ... 
101775    True
101776    True
101777    True
101778    True
101779    True
Name: raion, Length: 66956, dtype: bool

In [94]:
cian_rm[(cian_rm.raion.isna()) & (cian_rm.addr.apply(lambda x: 'р-н' in x.split(';')[2]))]

,city,raion,mikroraion,addr


In [100]:
cian_rm[(~cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: 'мкр' in x.split(';')[2]))]

,city,raion,mikroraion,addr
63536,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63537,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63538,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
63539,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
63540,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
...,...,...,...,...
68869,Новороссийск,NaN,Приморский,Краснодарский край; Новороссийск; мкр. Приморс...
68870,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
68871,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...
68872,Новороссийск,NaN,Южный,Краснодарский край; Новороссийск; мкр. Южный; ...


In [120]:
def rn_mkrn(addr_str):
    try:
        if 'мкр' in addr_str.split(';')[3]:
            return True
        else:
            return False
    except:
        if 'мкр' in addr_str.split(';')[2]:
            return True
        else:
            return False

In [121]:
'мкр' in cian_rm[(cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: mkr_3rd(x)))].iloc[100,-1].split(';')[3]

True

In [125]:
cian_rm[(cian_rm.mikroraion.isna()) & (cian_rm.addr.apply(lambda x: mkr_3rd(x)))].iloc[100,-1]

'Краснодарский край; Сочи; р-н Хостинский; Бытха мкр; улица Ясногорская; 16/6к3'

2023-03-24 15:26:41,032| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [76]:
cian_realty.head(5)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,210197059,5,None,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1
1,2,213575526,5,None,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1
2,2,232012292,5,None,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1
3,2,233965925,5,None,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1
4,2,239179225,5,None,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1


In [ ]:
st_time = datetime.now()
cian_realty.to_sql(name='realty', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)
print(len(realty_table), datetime.now() - st_time)

# dev prices cian

In [83]:
# realty_id

In [86]:
cian_all

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15


In [87]:
cian_realty.source_id.unique()[0]

2

In [84]:
current_source = cian_realty.source_id.unique()[0]
current_date = datetime.now().date()
id_df = get_realty_ids(sql_engine, current_date, current_source)[0]

NameError: name 'get_realty_ids' is not defined

In [277]:
id_df

,link,id
0,https://krasnodar.cian.ru/sale/flat/268501106/,32201
1,https://anapa.cian.ru/sale/flat/233965925/,32823
2,https://anapa.cian.ru/sale/flat/264443644/,33491
3,https://anapa.cian.ru/sale/flat/264569881/,33638
4,https://anapa.cian.ru/sale/flat/264931277/,33765
...,...,...
32579,https://krasnodar.cian.ru/sale/flat/274097524/,817839
32580,https://krasnodar.cian.ru/sale/flat/274097626/,817840
32581,https://krasnodar.cian.ru/sale/flat/274098241/,817841
32582,https://krasnodar.cian.ru/sale/flat/275302647/,817842


In [350]:
cian_prices = pd.DataFrame()

In [351]:
cian_prices['realty_id'] = cian_realty['link'].map(id_df.set_index('link')['id'])

In [352]:
cian_prices

,realty_id
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
101775,NaN
101776,NaN
101777,NaN
101778,NaN


In [353]:
# price

In [354]:
cian_prices['price'] = cian_all['price']

In [358]:
# price_sqrm

In [359]:
cian_all

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15


In [360]:
cian_prices['price_sqrm'] = cian_all['price_sqrm'].astype(float)

In [361]:
# date

In [362]:
cian_prices['date'] = datetime.now().date() # изменить на имя файла

In [363]:
datetime.now().date()

datetime.date(2023, 3, 22)

In [368]:
datetime.strptime('17-03-23', '%d-%m-%y').date()

datetime.date(2023, 3, 17)

In [372]:
cian_prices

,realty_id,price,price_sqrm,date
0,NaN,25000000,233645.0,2023-03-22
1,NaN,10100000,95283.0,2023-03-22
2,NaN,2100000,105000.0,2023-03-22
3,NaN,4500000,155172.0,2023-03-22
4,NaN,15300000,109286.0,2023-03-22
...,...,...,...,...
101775,NaN,1950000,108333.0,2023-03-22
101776,NaN,2000000,121212.0,2023-03-22
101777,NaN,4350000,90625.0,2023-03-22
101778,NaN,3680000,94359.0,2023-03-22


# write filename to txt

In [376]:
flist[:4]

['C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/01-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/02-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/04-03-23 (без дублей).csv',
 'C:/Users/pnknv/Documents/GitHub/yadisk_loader/saved_csv/05-03-23 (без дублей).csv']

In [443]:
with open('uploaded_files.txt', 'a+') as uploaded_txt:
    for line in flist[:3]:
        uploaded_txt.write(f"{line}\n")
uploaded_txt.close()

In [447]:
try:
    with open('uploaded_files.txt', 'r') as uploaded_txt:
        asd = uploaded_txt.read()
    uploaded_txt.close()
except:
    asd = []

In [448]:
asd

'07-02-2023\n07-02-23 (без дублей)\n08-02-23 (без дублей)\n09-02-23 (без дублей) 2\n09-02-23 (без дублей)\n10-02-23 (без дублей)\n11-02-23 (без дублей)\n12-02-23 (без дублей)\n13-02-23 (без дублей)\n14-02-23 (без дублей)\n15-02-23 (без дублей)\n16-02-23 (без дублей)\n17-02-23 (без дублей)\n18-02-23 (без дублей)\n19-02-23 (без дублей)\n19-02-23\n20-02-23 (без дублей)\n21-02-23 (без дублей)\n22-02-23 (без дублей)\n23-02-23 (без дублей)\n24-02-23 (без дублей)\n25-02-23 (без дублей)\n26-02-23 (без дублей)\n27-02-23 (без дублей)\n28-02-23 (без дублей)\n01-03-23 (без дублей)\n02-03-23 (без дублей)\n04-03-23 (без дублей)\n05-03-23 (без дублей)\n06-03-23 (без дублей)\n07-03-23 (без дублей)\n08-03-23 (без дублей)\n09-03-23 (без дублей)\n10-03-23 (без дублей)\n11-03-23 (без дублей)\n12-03-23 (без дублей)\n13-03-23 (без дублей)\n14-03-23 (без дублей)\n15-03-23 (без дублей)\n16-03-23 (без дублей)\n17-03-23 (без дублей)\n18-03-23 (без дублей)\n19-03-23 (без дублей)\n'

In [449]:
asd.split('\n')[:-1]

['07-02-2023',
 '07-02-23 (без дублей)',
 '08-02-23 (без дублей)',
 '09-02-23 (без дублей) 2',
 '09-02-23 (без дублей)',
 '10-02-23 (без дублей)',
 '11-02-23 (без дублей)',
 '12-02-23 (без дублей)',
 '13-02-23 (без дублей)',
 '14-02-23 (без дублей)',
 '15-02-23 (без дублей)',
 '16-02-23 (без дублей)',
 '17-02-23 (без дублей)',
 '18-02-23 (без дублей)',
 '19-02-23 (без дублей)',
 '19-02-23',
 '20-02-23 (без дублей)',
 '21-02-23 (без дублей)',
 '22-02-23 (без дублей)',
 '23-02-23 (без дублей)',
 '24-02-23 (без дублей)',
 '25-02-23 (без дублей)',
 '26-02-23 (без дублей)',
 '27-02-23 (без дублей)',
 '28-02-23 (без дублей)',
 '01-03-23 (без дублей)',
 '02-03-23 (без дублей)',
 '04-03-23 (без дублей)',
 '05-03-23 (без дублей)',
 '06-03-23 (без дублей)',
 '07-03-23 (без дублей)',
 '08-03-23 (без дублей)',
 '09-03-23 (без дублей)',
 '10-03-23 (без дублей)',
 '11-03-23 (без дублей)',
 '12-03-23 (без дублей)',
 '13-03-23 (без дублей)',
 '14-03-23 (без дублей)',
 '15-03-23 (без дублей)',
 '16-03-

# check all data

In [599]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_main']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

In [600]:
def close_sql_connection(server, engine):
    server.stop()
    engine.dispose()

In [601]:
def get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost,
                   localhost_port, database_username, database_password, database_name):
    
    sql_server = SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=(localhost, localhost_port)
    )

    sql_server.start()
    local_port = str(sql_server.local_bind_port)
    sql_engine = create_engine('mariadb+pymysql://{}:{}@{}:{}/{}'.format(database_username, database_password,
                                                                         localhost, local_port, database_name), 
                               poolclass=NullPool)

    return sql_server, sql_engine

In [602]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [603]:
def get_version(engine):
    last_version = f"SELECT * FROM realty WHERE source_id = 2 AND date = '2023-03-24'"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [604]:
api_version = get_version(sql_engine)[0]

In [605]:
close_sql_connection(sql_server, sql_engine)

In [606]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_test']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

In [607]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [608]:
def get_version(engine):
    last_version = "SELECT * FROM realty WHERE source_id = 2 AND date = '2023-03-24'"
    try:
        con_obj=engine.connect()
        last_version_db = pd.read_sql(text(last_version), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(exc)
        city_db = None
        exc_code = exc.code
    return last_version_db, exc_code

In [609]:
csv_version = get_version(sql_engine)[0]

In [610]:
close_sql_connection(sql_server, sql_engine)

# compare

In [631]:
csv_version

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
0,1630440,2,210197059,5.0,NaN,5,Краснодарский край; Анапа; улица Горького; 2А,107.0,7,12,https://anapa.cian.ru/sale/flat/210197059/,2023-03-24,None,270,Собственник,1,None
1,1630441,2,213575526,5.0,NaN,5,Краснодарский край; Анапа; улица Краснозеленых...,106.0,6,7,https://anapa.cian.ru/sale/flat/213575526/,2023-03-24,None,270,Риелтор,1,None
2,1630442,2,232012292,5.0,NaN,2,Краснодарский край; Анапа; улица Ленина; 194,20.0,1,9,https://anapa.cian.ru/sale/flat/232012292/,2023-03-24,None,270,Риелтор,1,None
3,1630443,2,233965925,5.0,NaN,3,Краснодарский край; Анапа; улица Толстого; 81,29.0,1,3,https://anapa.cian.ru/sale/flat/233965925/,2023-03-24,None,270,Агентство недвижимости,1,None
4,1630444,2,239179225,5.0,NaN,7,Краснодарский край; Анапа; Владимирская улица;...,140.0,13,13,https://anapa.cian.ru/sale/flat/239179225/,2023-03-24,None,270,Собственник,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103641,1734081,2,285278480,17.0,421.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,5,5,https://tomsk.cian.ru/sale/flat/285278480/,2023-03-24,None,270,Агентство недвижимости,1,None
103642,1734082,2,285278483,17.0,421.0,4,Томская область; Томск; р-н Октябрьский; мкр. ...,46.0,2,5,https://tomsk.cian.ru/sale/flat/285278483/,2023-03-24,None,270,Агентство недвижимости,1,None
103643,1734083,2,285278702,17.0,391.0,5,Томская область; Томск; р-н Октябрьский; улица...,51.0,2,5,https://tomsk.cian.ru/sale/flat/285278702/,2023-03-24,None,270,Агентство недвижимости,1,None
103644,1734084,2,285280917,17.0,384.0,2,Томская область; Томск; р-н Советский; Заливна...,19.6,1,4,https://tomsk.cian.ru/sale/flat/285280917/,2023-03-24,None,270,Агентство недвижимости,1,None


In [632]:
api_version

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
0,33638,2,264569881,5.0,NaN,3.0,Краснодарский край; Анапа; улица Поликарпова; 2к9,38.67,4,4,https://anapa.cian.ru/sale/flat/264569881/,2023-03-24,0,271,Агентство недвижимости,0,45921.0
1,34009,2,265377651,5.0,NaN,6.0,Краснодарский край; Анапа; улица Кати Соловьян...,122.00,4,7,https://anapa.cian.ru/sale/flat/265377651/,2023-03-24,0,271,Агентство недвижимости,0,46145.0
2,34194,2,265962891,5.0,NaN,3.0,Краснодарский край; Анапа; улица Объездная,44.50,10,12,https://anapa.cian.ru/sale/flat/265962891/,2023-03-24,0,271,Агентство недвижимости,0,45967.0
3,36225,2,268803650,5.0,NaN,5.0,Краснодарский край; Анапа; улица Ленина; 145,96.00,3,6,https://anapa.cian.ru/sale/flat/268803650/,2023-03-24,0,271,Агентство недвижимости,0,46061.0
4,41837,2,236679649,7.0,55.0,7.0,Челябинская область; Челябинск; р-н Калинински...,200.00,14,15,https://chelyabinsk.cian.ru/sale/flat/236679649/,2023-03-24,0,271,Собственник,0,3483.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12411,827244,2,285235730,3.0,373.0,4.0,Краснодарский край; Сочи; р-н Адлерский; Адлер...,43.00,5,8,https://sochi.cian.ru/sale/flat/285235730/,2023-03-24,0,271,Агентство недвижимости,0,NaN
12412,827250,2,283513138,17.0,387.0,4.0,Томская область; Томск; р-н Кировский; Стрижи ЖК,59.39,6,22,https://tomsk.cian.ru/sale/flat/283513138/,2023-03-24,1,271,Застройщик,0,80786.0
12413,827302,2,285227600,17.0,405.0,3.0,Томская область; Томск; р-н Октябрьский; мкр. ...,28.10,3,4,https://tomsk.cian.ru/sale/flat/285227600/,2023-03-24,1,271,None,0,34821.0
12414,827306,2,285228122,17.0,394.0,4.0,Томская область; Томск; р-н Ленинский; Черемош...,50.00,6,6,https://tomsk.cian.ru/sale/flat/285228122/,2023-03-24,1,271,Агентство недвижимости,0,NaN


In [633]:
csvinapi_version = csv_version[csv_version.link.isin(api_version.link.to_list())]

In [614]:
csvinapi_version

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
83147,1713587,2,280277793,3.0,308.0,4,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.30,4,19,https://sochi.cian.ru/sale/flat/280277793/,2023-03-24,None,270,Агентство недвижимости,1,None
83148,1713588,2,280278783,3.0,290.0,3,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.00,7,9,https://sochi.cian.ru/sale/flat/280278783/,2023-03-24,None,270,Риелтор,1,None
83150,1713590,2,280279971,3.0,303.0,4,Краснодарский край; Сочи; р-н Центральный; мкр...,36.20,8,12,https://sochi.cian.ru/sale/flat/280279971/,2023-03-24,None,270,Агентство недвижимости,1,None
83151,1713591,2,280280095,NaN,NaN,5,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,2023-03-24,None,270,Агентство недвижимости,1,None
83152,1713592,2,280280191,NaN,NaN,2,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,2023-03-24,None,270,Агентство недвижимости,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83395,1713835,2,280322081,3.0,285.0,2,Краснодарский край; Сочи; р-н Центральный; мкр...,60.00,8,15,https://sochi.cian.ru/sale/flat/280322081/,2023-03-24,None,270,Агентство недвижимости,1,None
83396,1713836,2,280322258,3.0,293.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,139.50,18,26,https://sochi.cian.ru/sale/flat/280322258/,2023-03-24,None,270,Агентство недвижимости,1,None
83397,1713837,2,280322346,NaN,NaN,3,Краснодарский край; Сочи городской округ; Крас...,45.13,7,9,https://sochi.cian.ru/sale/flat/280322346/,2023-03-24,None,270,Агентство недвижимости,1,None
83398,1713838,2,280322509,NaN,NaN,4,Краснодарский край; Сочи городской округ; Крас...,73.10,5,9,https://sochi.cian.ru/sale/flat/280322509/,2023-03-24,None,270,Агентство недвижимости,1,None


In [634]:
csvinapi_version

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
83147,1713587,2,280277793,3.0,308.0,4,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.30,4,19,https://sochi.cian.ru/sale/flat/280277793/,2023-03-24,None,270,Агентство недвижимости,1,None
83148,1713588,2,280278783,3.0,290.0,3,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.00,7,9,https://sochi.cian.ru/sale/flat/280278783/,2023-03-24,None,270,Риелтор,1,None
83150,1713590,2,280279971,3.0,303.0,4,Краснодарский край; Сочи; р-н Центральный; мкр...,36.20,8,12,https://sochi.cian.ru/sale/flat/280279971/,2023-03-24,None,270,Агентство недвижимости,1,None
83151,1713591,2,280280095,NaN,NaN,5,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,2023-03-24,None,270,Агентство недвижимости,1,None
83152,1713592,2,280280191,NaN,NaN,2,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,2023-03-24,None,270,Агентство недвижимости,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83395,1713835,2,280322081,3.0,285.0,2,Краснодарский край; Сочи; р-н Центральный; мкр...,60.00,8,15,https://sochi.cian.ru/sale/flat/280322081/,2023-03-24,None,270,Агентство недвижимости,1,None
83396,1713836,2,280322258,3.0,293.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,139.50,18,26,https://sochi.cian.ru/sale/flat/280322258/,2023-03-24,None,270,Агентство недвижимости,1,None
83397,1713837,2,280322346,NaN,NaN,3,Краснодарский край; Сочи городской округ; Крас...,45.13,7,9,https://sochi.cian.ru/sale/flat/280322346/,2023-03-24,None,270,Агентство недвижимости,1,None
83398,1713838,2,280322509,NaN,NaN,4,Краснодарский край; Сочи городской округ; Крас...,73.10,5,9,https://sochi.cian.ru/sale/flat/280322509/,2023-03-24,None,270,Агентство недвижимости,1,None


In [635]:
csvinapi_version.type_id = csvinapi_version.type_id.astype(float)

In [636]:
api_215 = api_version[api_version.link.isin(csv_version.link.to_list())]

In [637]:
csvinapi_version.reset_index(drop=True, inplace=True)
api_215.reset_index(drop=True, inplace=True)

In [ ]:
'city_id','district_id',

In [646]:
csv_comp = csvinapi_version[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']]
api_comp = api_215[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']]

In [647]:
csv_comp = csv_comp.sort_values('ad_id').reset_index(drop=True)
api_comp = api_comp.sort_values('ad_id').reset_index(drop=True)

In [648]:
csv_comp == api_comp

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
0,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True
3,True,False,False,True,True,True,True,True,True,True
4,True,False,False,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
247,True,True,True,True,True,True,True,True,True,True
248,True,True,True,True,True,True,True,True,True,True
249,True,False,False,True,True,True,True,True,True,True
250,True,False,False,True,True,True,True,True,True,True


In [649]:
api_comp.columns

Index(['ad_id', 'city_id', 'district_id', 'type_id', 'addr', 'square', 'floor',
       'house_floors', 'link', 'offer_from'],
      dtype='object')

In [663]:
api_comp.city_id != csv_comp.city_id

0      False
1      False
2      False
3       True
4       True
       ...  
247    False
248    False
249     True
250     True
251    False
Name: city_id, Length: 252, dtype: bool

In [666]:
csv_comp[api_comp.city_id != csv_comp.city_id].head(5)

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
3,280280095,NaN,NaN,5.0,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,Агентство недвижимости
4,280280191,NaN,NaN,2.0,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,Агентство недвижимости
5,280280286,NaN,NaN,4.0,Краснодарский край; Сочи городской округ; Крас...,76.00,3,4,https://sochi.cian.ru/sale/flat/280280286/,Агентство недвижимости
11,280281525,NaN,NaN,3.0,Краснодарский край; Сочи городской округ; Даго...,37.36,20,20,https://sochi.cian.ru/sale/flat/280281525/,Агентство недвижимости
12,280281526,NaN,NaN,3.0,Краснодарский край; Сочи городской округ; Даго...,29.92,20,20,https://sochi.cian.ru/sale/flat/280281526/,Агентство недвижимости


In [665]:
api_comp[api_comp.city_id != csv_comp.city_id].head(5)

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
3,280280095,NaN,NaN,5.0,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,Агентство недвижимости
4,280280191,NaN,NaN,2.0,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,Агентство недвижимости
5,280280286,NaN,NaN,4.0,Краснодарский край; Сочи городской округ; Крас...,76.00,3,4,https://sochi.cian.ru/sale/flat/280280286/,Агентство недвижимости
11,280281525,NaN,NaN,3.0,Краснодарский край; Сочи городской округ; Даго...,37.36,20,20,https://sochi.cian.ru/sale/flat/280281525/,Агентство недвижимости
12,280281526,NaN,NaN,3.0,Краснодарский край; Сочи городской округ; Даго...,29.92,20,20,https://sochi.cian.ru/sale/flat/280281526/,Агентство недвижимости


In [652]:
for i in api_comp.columns:
    if api_comp[i].to_list() == csv_comp[i].to_list():
        print(i, 'equal')
    else:
        print(i)

ad_id equal
city_id
district_id
type_id
addr equal
square equal
floor equal
house_floors equal
link equal
offer_from equal


In [656]:
csv_comp.city_id.value_counts()

3.0    218
Name: city_id, dtype: int64

In [655]:
api_comp.city_id.value_counts()

3.0    218
Name: city_id, dtype: int64

In [ ]:
city_id district_id type_id

In [645]:
csv_comp[csv_comp == api_comp]

,ad_id,type_id,addr,square,floor,house_floors,link,offer_from
0,280277793,4.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.30,4,19,https://sochi.cian.ru/sale/flat/280277793/,Агентство недвижимости
1,280278783,3.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.00,7,9,https://sochi.cian.ru/sale/flat/280278783/,Риелтор
2,280279971,4.0,Краснодарский край; Сочи; р-н Центральный; мкр...,36.20,8,12,https://sochi.cian.ru/sale/flat/280279971/,Агентство недвижимости
3,280280095,5.0,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,Агентство недвижимости
4,280280191,2.0,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,Агентство недвижимости
...,...,...,...,...,...,...,...,...
247,280322081,2.0,Краснодарский край; Сочи; р-н Центральный; мкр...,60.00,8,15,https://sochi.cian.ru/sale/flat/280322081/,Агентство недвижимости
248,280322258,5.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,139.50,18,26,https://sochi.cian.ru/sale/flat/280322258/,Агентство недвижимости
249,280322346,3.0,Краснодарский край; Сочи городской округ; Крас...,45.13,7,9,https://sochi.cian.ru/sale/flat/280322346/,Агентство недвижимости
250,280322509,4.0,Краснодарский край; Сочи городской округ; Крас...,73.10,5,9,https://sochi.cian.ru/sale/flat/280322509/,Агентство недвижимости


In [628]:
csv_comp.head(20)

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
0,280277793,3.0,308.0,4,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.30,4,19,https://sochi.cian.ru/sale/flat/280277793/,Агентство недвижимости
1,280278783,3.0,290.0,3,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.00,7,9,https://sochi.cian.ru/sale/flat/280278783/,Риелтор
2,280279971,3.0,303.0,4,Краснодарский край; Сочи; р-н Центральный; мкр...,36.20,8,12,https://sochi.cian.ru/sale/flat/280279971/,Агентство недвижимости
3,280280095,NaN,NaN,5,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,Агентство недвижимости
4,280280191,NaN,NaN,2,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,Агентство недвижимости
5,280280286,NaN,NaN,4,Краснодарский край; Сочи городской округ; Крас...,76.00,3,4,https://sochi.cian.ru/sale/flat/280280286/,Агентство недвижимости
6,280280384,3.0,295.0,3,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,27.00,2,2,https://sochi.cian.ru/sale/flat/280280384/,Агентство недвижимости
7,280280895,3.0,290.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,90.00,7,23,https://sochi.cian.ru/sale/flat/280280895/,Агентство недвижимости
8,280281118,3.0,291.0,4,Краснодарский край; Сочи; р-н Центральный; мкр...,30.00,2,3,https://sochi.cian.ru/sale/flat/280281118/,Агентство недвижимости
9,280281521,3.0,285.0,5,Краснодарский край; Сочи; р-н Центральный; мкр...,75.00,4,5,https://sochi.cian.ru/sale/flat/280281521/,Агентство недвижимости


In [629]:
api_comp.head(20)

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
0,280277793,3.0,308.0,4.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.30,4,19,https://sochi.cian.ru/sale/flat/280277793/,Агентство недвижимости
1,280278783,3.0,290.0,3.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.00,7,9,https://sochi.cian.ru/sale/flat/280278783/,Риелтор
2,280279971,3.0,303.0,4.0,Краснодарский край; Сочи; р-н Центральный; мкр...,36.20,8,12,https://sochi.cian.ru/sale/flat/280279971/,Агентство недвижимости
3,280280095,NaN,NaN,5.0,Краснодарский край; Сочи городской округ; с. Э...,62.00,1,4,https://sochi.cian.ru/sale/flat/280280095/,Агентство недвижимости
4,280280191,NaN,NaN,2.0,Краснодарский край; Сочи городской округ; с. Э...,44.00,5,6,https://sochi.cian.ru/sale/flat/280280191/,Агентство недвижимости
5,280280286,NaN,NaN,4.0,Краснодарский край; Сочи городской округ; Крас...,76.00,3,4,https://sochi.cian.ru/sale/flat/280280286/,Агентство недвижимости
6,280280384,3.0,295.0,3.0,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,27.00,2,2,https://sochi.cian.ru/sale/flat/280280384/,Агентство недвижимости
7,280280895,3.0,290.0,5.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,90.00,7,23,https://sochi.cian.ru/sale/flat/280280895/,Агентство недвижимости
8,280281118,3.0,291.0,4.0,Краснодарский край; Сочи; р-н Центральный; мкр...,30.00,2,3,https://sochi.cian.ru/sale/flat/280281118/,Агентство недвижимости
9,280281521,3.0,285.0,5.0,Краснодарский край; Сочи; р-н Центральный; мкр...,75.00,4,5,https://sochi.cian.ru/sale/flat/280281521/,Агентство недвижимости


In [396]:
pd.concat([csvinapi_version[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']],api_215[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']]]).drop_duplicates()

,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,offer_from
0,280277793,3,374.0,4,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.3,4,19,https://sochi.cian.ru/sale/flat/280277793/,Агентство недвижимости
1,280278783,3,374.0,3,Краснодарский край; Сочи; р-н Хостинский; мкр....,42.0,7,9,https://sochi.cian.ru/sale/flat/280278783/,Риелтор
2,280279971,3,372.0,4,Краснодарский край; Сочи; р-н Центральный; мкр...,36.2,8,12,https://sochi.cian.ru/sale/flat/280279971/,Агентство недвижимости
3,280280384,3,370.0,3,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,27.0,2,2,https://sochi.cian.ru/sale/flat/280280384/,Агентство недвижимости
4,280280895,3,374.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,90.0,7,23,https://sochi.cian.ru/sale/flat/280280895/,Агентство недвижимости
...,...,...,...,...,...,...,...,...,...,...
209,280284749,3,287.0,5,Краснодарский край; Сочи; р-н Центральный; мкр...,80.0,3,11,https://sochi.cian.ru/sale/flat/280284749/,Агентство недвижимости
210,280284810,3,308.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,70.0,5,20,https://sochi.cian.ru/sale/flat/280284810/,Агентство недвижимости
211,280285889,3,299.0,3,Краснодарский край; Сочи; р-н Центральный; мкр...,36.0,4,5,https://sochi.cian.ru/sale/flat/280285889/,Агентство недвижимости
212,280286444,3,293.0,5,Краснодарский край; Сочи; р-н Хостинский; мкр....,90.0,7,26,https://sochi.cian.ru/sale/flat/280286444/,Агентство недвижимости


In [391]:
csvinapi_version[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']].compare(api_215[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']])

ad_id              district_id        type_id        \
            self        other        self  other    self other   
0            NaN          NaN       374.0  308.0     NaN   NaN   
1    280278783.0  280279971.0       374.0  303.0     3.0   4.0   
2    280279971.0  280295425.0       372.0  303.0     NaN   NaN   
3    280280384.0  280307879.0       370.0  298.0     3.0   4.0   
4    280280895.0  280307953.0       374.0  321.0     5.0   4.0   
..           ...          ...         ...    ...     ...   ...   
209  280315459.0  280284749.0       372.0  287.0     4.0   5.0   
210  280317376.0  280284810.0       372.0  308.0     4.0   5.0   
211  280322081.0  280285889.0       372.0  299.0     2.0   3.0   
212  280322258.0  280286444.0       374.0  293.0     NaN   NaN   
213  280322540.0  280308604.0       370.0  294.0     4.0   3.0   

                                                  addr  \
                                                  self   
0                                                  NaN   
1    Краснодарский край; Сочи; р-н Хостинский; мкр....   
2    Краснодарский край; Сочи; р-н Центральный; мкр...   
3    Краснодарский край; Сочи; р-н Адлерский; мкр. ...   
4    Краснодарский край; Сочи; р-н Хостинский; мкр....   
..                                                 ...   
209  Краснодарский край; Сочи; р-н Центральный; мкр...   
210  Краснодарский край; Сочи; р-н Центральный; мкр...   
211  Краснодарский край; Сочи; р-н Центральный; мкр...   
212                                                NaN   
213  Краснодарский край; Сочи; р-н Адлерский; мкр. ...   

                                                       square       floor  \
                                                 other   self other  self   
0                                                  NaN    NaN   NaN   NaN   
1    Краснодарский край; Сочи; р-н Центральный; мкр...   42.0  36.2   7.0   
2    Краснодарский край; Сочи; р-н Центральный; мкр...   36.2  64.0   8.0   
3    Краснодарский край; Сочи; р-н Адлерский; мкр. ...   27.0  39.0   NaN   
4    Краснодарский край; Сочи; р-н Адлерский; мкр. ...   90.0  47.0   7.0   
..                                                 ...    ...   ...   ...   
209  Краснодарский край; Сочи; р-н Центральный; мкр...   48.0  80.0   NaN   
210  Краснодарский край; Сочи; р-н Хостинский; мкр....   53.4  70.0   4.0   
211  Краснодарский край; Сочи; р-н Центральный; мкр...   60.0  36.0   8.0   
212                                                NaN  139.5  90.0  18.0   
213  Краснодарский край; Сочи; р-н Адлерский; мкр. ...   34.1  45.0   NaN   

          house_floors                                              link  \
    other         self other                                        self   
0     NaN          NaN   NaN                                         NaN   
1     8.0          9.0  12.0  https://sochi.cian.ru/sale/flat/280278783/   
2     4.0         12.0   5.0  https://sochi.cian.ru/sale/flat/280279971/   
3     NaN          2.0   3.0  https://sochi.cian.ru/sale/flat/280280384/   
4     3.0         23.0   5.0  https://sochi.cian.ru/sale/flat/280280895/   
..    ...          ...   ...                                         ...   
209   NaN          5.0  11.0  https://sochi.cian.ru/sale/flat/280315459/   
210   5.0          5.0  20.0  https://sochi.cian.ru/sale/flat/280317376/   
211   4.0         15.0   5.0  https://sochi.cian.ru/sale/flat/280322081/   
212   7.0          NaN   NaN  https://sochi.cian.ru/sale/flat/280322258/   
213   NaN          4.0   7.0  https://sochi.cian.ru/sale/flat/280322540/   

                                                  offer_from  \
                                          other         self   
0                                           NaN          NaN   
1    https://sochi.cian.ru/sale/flat/280279971/      Риелтор   
2    https://sochi.cian.ru/sale/flat/280295425/          NaN   
3    https://sochi.cian.ru/sale/flat/280307879/          NaN   
4    https://sochi.cia

In [373]:
intersect = 0
for i,val in csvinapi_version.iterrows():
    for j, val2 in api_215.iterrows():
        if val[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']] == val2[['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']]:
            intersect += 1

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [375]:
api_215.iloc[0,:][['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']] == api_215.iloc[0,:][['ad_id','city_id','district_id','type_id','addr','square','floor','house_floors','link','offer_from']]

ad_id           True
city_id         True
district_id     True
type_id         True
addr            True
square          True
floor           True
house_floors    True
link            True
offer_from      True
Name: 1205, dtype: bool

In [352]:
api_215[api_215.type_id.isna()]

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
6576,781527,2,280281763,3,285.0,NaN,Краснодарский край; Сочи; р-н Центральный; мкр...,175.7,2,4,https://sochi.cian.ru/sale/flat/280281763/,2023-03-24,0,271,Агентство недвижимости,0,78913.0


In [350]:
api_215api_215.dropna(subset=['type_id'])

,id,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,house_id
1205,470040,2,280277793,3,308.0,4.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,62.3,4,19,https://sochi.cian.ru/sale/flat/280277793/,2023-03-24,0,271,Агентство недвижимости,0,8126.0
1206,470064,2,280279971,3,303.0,4.0,Краснодарский край; Сочи; р-н Центральный; мкр...,36.2,8,12,https://sochi.cian.ru/sale/flat/280279971/,2023-03-24,0,271,Агентство недвижимости,0,7708.0
1232,473215,2,280295425,3,303.0,4.0,Краснодарский край; Сочи; р-н Центральный; мкр...,64.0,4,5,https://sochi.cian.ru/sale/flat/280295425/,2023-03-24,0,271,Агентство недвижимости,0,9190.0
1233,473303,2,280307879,3,298.0,4.0,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,39.0,2,3,https://sochi.cian.ru/sale/flat/280307879/,2023-03-24,0,271,Агентство недвижимости,0,9225.0
1234,473304,2,280307953,3,321.0,4.0,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,47.0,3,5,https://sochi.cian.ru/sale/flat/280307953/,2023-03-24,0,271,Агентство недвижимости,0,8052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8029,814146,2,280284749,3,287.0,5.0,Краснодарский край; Сочи; р-н Центральный; мкр...,80.0,3,11,https://sochi.cian.ru/sale/flat/280284749/,2023-03-24,0,271,Агентство недвижимости,0,8333.0
8030,814147,2,280284810,3,308.0,5.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,70.0,5,20,https://sochi.cian.ru/sale/flat/280284810/,2023-03-24,0,271,Агентство недвижимости,0,7743.0
8031,814148,2,280285889,3,299.0,3.0,Краснодарский край; Сочи; р-н Центральный; мкр...,36.0,4,5,https://sochi.cian.ru/sale/flat/280285889/,2023-03-24,0,271,Агентство недвижимости,0,7828.0
8032,814149,2,280286444,3,293.0,5.0,Краснодарский край; Сочи; р-н Хостинский; мкр....,90.0,7,26,https://sochi.cian.ru/sale/flat/280286444/,2023-03-24,0,271,Агентство недвижимости,0,76995.0


In [343]:
api_215.type_id = api_215.type_id.astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [496]:
def checkk():
    for i in range(7):
        print(i)
        if i == 3:
            print(i, i, i)
            continue
        elif i == 5:
            print(i, i, i)
            continue
        else:
            continue
    if i == 6:
        print(6)
        


In [497]:
checkk()

0
1
2
3
3 3 3
4
5
5 5 5
6
6


In [480]:
asdasd = pd.read_csv('23-03-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [482]:
asdasd[asdasd['Заголовок'].isna()]

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
14316,https://www.avito.ru/kazan/kvartiry/_2366206291,NaN,NaN,NaN,NaN,6100000,103918.0,NaN,NaN,"ул. Братьев Касимовых, 60",1 неделю назад,Агентство,частное лицо,https://www.avito.ru/user/34509eaa888e0dab6a8d...,78


In [ ]:
asdasd[asdasd.addr.isna()]

In [667]:
asdasd = pd.read_csv('24-03-23.csv', delimiter=',', encoding='cp1251')

In [668]:
asdasd

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103641,cian,285278480,Томская область; Томск; р-н Октябрьский; мкр. ...,Томск,Октябрьский,3-й,46.0,5,5,2 комн,https://tomsk.cian.ru/sale/flat/285278480/,3890000,84565,Агентство недвижимости,2023-03-24 13:05:02
103642,cian,285278483,Томская область; Томск; р-н Октябрьский; мкр. ...,Томск,Октябрьский,3-й,46.0,2,5,2 комн,https://tomsk.cian.ru/sale/flat/285278483/,4000000,86957,Агентство недвижимости,2023-03-24 13:05:03
103643,cian,285278702,Томская область; Томск; р-н Октябрьский; улица...,Томск,Октябрьский,NaN,51.0,2,5,3 комн,https://tomsk.cian.ru/sale/flat/285278702/,3650000,71569,Агентство недвижимости,2023-03-24 13:09:03
103644,cian,285280917,Томская область; Томск; р-н Советский; Заливна...,Томск,Советский,NaN,19.6,1,4,Студия,https://tomsk.cian.ru/sale/flat/285280917/,2650000,135204,Агентство недвижимости,2023-03-24 13:46:03


In [669]:
asdasd[asdasd.link.isin(api_version.link.to_list())]

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
83147,cian,280277793,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Яна Фабрициуса,62.30,4,19,2 комн,https://sochi.cian.ru/sale/flat/280277793/,30975000,497191,Агентство недвижимости,2022-11-18 10:37:54
83148,cian,280278783,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Бытха,42.00,7,9,1 комн,https://sochi.cian.ru/sale/flat/280278783/,10250000,244048,Риелтор,2022-11-18 10:52:25
83150,cian,280279971,Краснодарский край; Сочи; р-н Центральный; мкр...,Сочи,Центральный,Завокзальный,36.20,8,12,2 комн,https://sochi.cian.ru/sale/flat/280279971/,11000000,303867,Агентство недвижимости,2022-11-18 11:24:10
83151,cian,280280095,Краснодарский край; Сочи городской округ; с. Э...,Эсто-Садок,NaN,NaN,62.00,1,4,3 комн,https://sochi.cian.ru/sale/flat/280280095/,20000000,322581,Агентство недвижимости,2022-11-18 11:27:59
83152,cian,280280191,Краснодарский край; Сочи городской округ; с. Э...,Эсто-Садок,NaN,NaN,44.00,5,6,Студия,https://sochi.cian.ru/sale/flat/280280191/,8500000,193182,Агентство недвижимости,2022-11-18 11:31:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83395,cian,280322081,Краснодарский край; Сочи; р-н Центральный; мкр...,Сочи,Центральный,Центр,60.00,8,15,Студия,https://sochi.cian.ru/sale/flat/280322081/,18000000,300000,Агентство недвижимости,2022-11-20 12:39:15
83396,cian,280322258,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Приморье,139.50,18,26,3 комн,https://sochi.cian.ru/sale/flat/280322258/,35500000,254480,Агентство недвижимости,2022-11-20 12:56:16
83397,cian,280322346,Краснодарский край; Сочи городской округ; Крас...,Красная Поляна,NaN,NaN,45.13,7,9,1 комн,https://sochi.cian.ru/sale/flat/280322346/,6075000,134611,Агентство недвижимости,2022-11-20 13:03:50
83398,cian,280322509,Краснодарский край; Сочи городской округ; Крас...,Красная Поляна,NaN,NaN,73.10,5,9,2 комн,https://sochi.cian.ru/sale/flat/280322509/,9868500,135000,Агентство недвижимости,2022-11-20 13:22:46


In [484]:
'slowpokalipsis@gmail.com' == 'slowpokalipsis@gmail.com'

True

In [486]:
asdasd[asdasd.addr.apply(lambda x: 'Адлерский' in x)]

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
28619,cian,270222384,Краснодарский край; Сириус пгт; р-н Адлерский;...,Сириус,Адлерский,NaN,127.0,2,3,4 комн,https://krasnodar.cian.ru/sale/flat/270222384/,40000000,314961,Агентство недвижимости,2022-02-16 12:16:55
30808,cian,274046257,Краснодарский край; Сириус пгт; р-н Адлерский;...,Сириус,Адлерский,NaN,20.0,1,4,1 комн,https://krasnodar.cian.ru/sale/flat/274046257/,11000000,550000,Риелтор,2022-05-28 21:39:39
31058,cian,274438921,Краснодарский край; Сириус пгт; р-н Адлерский;...,Сириус,Адлерский,NaN,110.0,1,5,3 комн,https://krasnodar.cian.ru/sale/flat/274438921/,63000000,572727,Агентство недвижимости,2022-06-07 13:43:21
42559,cian,281178147,Краснодарский край; Сириус пгт; р-н Адлерский;...,Сириус,Адлерский,NaN,64.0,5,5,2 комн,https://krasnodar.cian.ru/sale/flat/281178147/,45000000,703125,Агентство недвижимости,2022-12-10 11:16:58
43831,cian,281739171,Краснодарский край; Сириус пгт; р-н Адлерский;...,Сириус,Адлерский,NaN,118.0,5,5,5 комн,https://krasnodar.cian.ru/sale/flat/281739171/,15800000,133898,Агентство недвижимости,2022-12-26 10:01:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98413,cian,285272871,Краснодарский край; Сочи; р-н Адлерский; улица...,Сочи,Адлерский,NaN,44.0,2,4,2 комн,https://sochi.cian.ru/sale/flat/285272871/,10500000,238636,Риелтор,2023-03-24 11:17:44
98415,cian,285273007,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,Сочи,Адлерский,Голубые дали,37.0,10,16,1 комн,https://sochi.cian.ru/sale/flat/285273007/,12500000,337838,Агентство недвижимости,2023-03-24 11:22:36
98419,cian,285273649,Краснодарский край; Сочи; р-н Адлерский; улица...,Сочи,Адлерский,NaN,35.0,1,3,1 комн,https://sochi.cian.ru/sale/flat/285273649/,5000000,142857,Агентство недвижимости,2023-03-24 11:38:43
98427,cian,285275035,Краснодарский край; Сочи; р-н Адлерский; мкр. ...,Сочи,Адлерский,Адлер Центр,24.2,1,4,Студия,https://sochi.cian.ru/sale/flat/285275035/,9900000,409091,Агентство недвижимости,2023-03-24 12:07:34


In [479]:
asdasd[asdasd.addr.isna()]

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date


In [337]:
asdasd[asdasd.link.isin(api_version.link.to_list())]

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
83147,cian,280277793,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Яна Фабрициуса,62.30,4,19,2 комн,https://sochi.cian.ru/sale/flat/280277793/,30975000,497191,Агентство недвижимости,2022-11-18 10:37:54
83148,cian,280278783,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Бытха,42.00,7,9,1 комн,https://sochi.cian.ru/sale/flat/280278783/,10250000,244048,Риелтор,2022-11-18 10:52:25
83150,cian,280279971,Краснодарский край; Сочи; р-н Центральный; мкр...,Сочи,Центральный,Завокзальный,36.20,8,12,2 комн,https://sochi.cian.ru/sale/flat/280279971/,11000000,303867,Агентство недвижимости,2022-11-18 11:24:10
83151,cian,280280095,Краснодарский край; Сочи городской округ; с. Э...,Эсто-Садок,NaN,NaN,62.00,1,4,3 комн,https://sochi.cian.ru/sale/flat/280280095/,20000000,322581,Агентство недвижимости,2022-11-18 11:27:59
83152,cian,280280191,Краснодарский край; Сочи городской округ; с. Э...,Эсто-Садок,NaN,NaN,44.00,5,6,Студия,https://sochi.cian.ru/sale/flat/280280191/,8500000,193182,Агентство недвижимости,2022-11-18 11:31:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83395,cian,280322081,Краснодарский край; Сочи; р-н Центральный; мкр...,Сочи,Центральный,Центр,60.00,8,15,Студия,https://sochi.cian.ru/sale/flat/280322081/,18000000,300000,Агентство недвижимости,2022-11-20 12:39:15
83396,cian,280322258,Краснодарский край; Сочи; р-н Хостинский; мкр....,Сочи,Хостинский,Приморье,139.50,18,26,3 комн,https://sochi.cian.ru/sale/flat/280322258/,35500000,254480,Агентство недвижимости,2022-11-20 12:56:16
83397,cian,280322346,Краснодарский край; Сочи городской округ; Крас...,Красная Поляна,NaN,NaN,45.13,7,9,1 комн,https://sochi.cian.ru/sale/flat/280322346/,6075000,134611,Агентство недвижимости,2022-11-20 13:03:50
83398,cian,280322509,Краснодарский край; Сочи городской округ; Крас...,Красная Поляна,NaN,NaN,73.10,5,9,2 комн,https://sochi.cian.ru/sale/flat/280322509/,9868500,135000,Агентство недвижимости,2022-11-20 13:22:46


In [166]:
# district_id

In [173]:
district_df['len_addr'] = district_df.addr.apply(lambda x: len(x.split(';')))

In [174]:
district_df['len_addr_coma'] = district_df.addr.apply(lambda x: len(x.split(',')))

In [180]:
district_df.len_addr.value_counts()

6    20113
5    10660
7     6324
1     3610
4     1556
3      135
8      116
2        1
Name: len_addr, dtype: int64

In [185]:
random.randint(0,3610)

1091

In [190]:
district_df[(district_df.city_id == 5)]

,city_id,district_id,addr,len_addr,len_addr_coma
15761,5,NaN,Краснодарский край; Анапа,2,1
15762,5,NaN,Краснодарский край; Анапа; 12-й мкр,3,1
15763,5,NaN,Краснодарский край; Анапа; 12-й мкр; 10,4,1
15764,5,NaN,Краснодарский край; Анапа; 12-й мкр; 11,4,1
15765,5,NaN,Краснодарский край; Анапа; 12-й мкр; 12,4,1
...,...,...,...,...,...
16837,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 47к3,4,1
16838,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к1,4,1
16839,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к2,4,1
16840,5,NaN,Краснодарский край; Анапа; шоссе Супсехское; 4к3,4,1


In [345]:
# district_df[district_df.len_addr == 1].iloc[[random.randint(0,3610) for i in range(50)],:]